In [ ]:
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
from rasterio import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.features import geometry_mask
import pandas as pd
from scipy.spatial import cKDTree
from sklearn.preprocessing import MinMaxScaler
from shapely.geometry import Point, Polygon, box
from tqdm import tqdm
import json
import os
import warnings
from scipy.stats import wasserstein_distance
from typing import Dict, List, Tuple
import numpy as np

warnings.filterwarnings('ignore')

# Define target CRS for all analysis (UTM Zone 30N for Ghana/Accra)
TARGET_CRS = "EPSG:32630"

In [ ]:
from typing import Dict, Tuple, List
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy.spatial import cKDTree
from scipy.stats import wasserstein_distance
from shapely.geometry import box, Point
import os
import json
from pathlib import Path

# GBA settings
TARGET_CRS = "EPSG:32630"  # UTM zone 30N for Accra, Ghana

class TODQAAssessor:
    """
    Task-Oriented Data Quality Assessment (Li et al., 2019)
    Modified for urban building datasets
    """
    
    def __init__(self, k=16, random_state=42):
        self.k = k  # number of LSH hyperplanes
        self.hyperplanes = None
        self.random_state = random_state
        np.random.seed(random_state)
    
    def fit_hyperplanes(self, feature_dim: int):
        """Initialize k random hyperplanes for LSH"""
        self.hyperplanes = np.random.randn(self.k, feature_dim)
    
    def hash_vector(self, x: np.ndarray) -> np.ndarray:
        """Compute LSH hash for a feature vector"""
        return (np.dot(self.hyperplanes, x) >= 0).astype(int)
    
    def compute_relevancy(self, D_features: np.ndarray, S_features: np.ndarray, 
                         task_type: str = 'accessibility') -> float:
        """
        Compute task relevancy score q_D|S_tr
        
        Parameters:
        -----------
        D_features : array-like, feature vectors from Open Buildings dataset
        S_features : array-like, feature vectors from task sample
        task_type : str, type of similarity function to use
        
        Returns:
        --------
        q_score : float, task relevancy score (0-1)
        """
        
        if len(D_features) == 0 or len(S_features) == 0:
            return 0.0
            
        if self.hyperplanes is None:
            self.fit_hyperplanes(D_features.shape[1])
        
        # Initialize buckets
        buckets = {}
        
        # Hash D features
        for i, x in enumerate(D_features):
            hash_val = tuple(self.hash_vector(x))
            bucket = buckets.setdefault(hash_val, {'D': [], 'S': []})
            bucket['D'].append(x)
        
        # Hash S features
        for i, x in enumerate(S_features):
            hash_val = tuple(self.hash_vector(x))
            bucket = buckets.setdefault(hash_val, {'D': [], 'S': []})
            bucket['S'].append(x)
        
        # Compute bucket-level similarities
        total_similarity = 0.0
        total_weight = 0
        
        for bucket_data in buckets.values():
            D_prime = np.array(bucket_data['D'])
            S_prime = np.array(bucket_data['S'])
            
            if len(D_prime) > 0 and len(S_prime) > 0:
                # Compute similarity based on task type
                if task_type == 'accessibility':
                    similarity = self._accessibility_similarity(D_prime, S_prime)
                elif task_type == 'morphology':
                    similarity = self._morphology_similarity(D_prime, S_prime)
                elif task_type == 'density':
                    similarity = self._density_similarity(D_prime, S_prime)
                else:
                    raise ValueError(f"Unknown task type: {task_type}")
                
                total_similarity += similarity * len(D_prime)
                total_weight += len(D_prime)
        
        return total_similarity / total_weight if total_weight > 0 else 0.0
    
    def _accessibility_similarity(self, D_prime: np.ndarray, S_prime: np.ndarray) -> float:
        """Compute accessibility similarity in a bucket"""
        if len(D_prime) == 0 or len(S_prime) == 0:
            return 0.0
        
        # Use Earth Mover's Distance (Wasserstein distance)
        D_flat = D_prime.flatten()
        S_flat = S_prime.flatten()
        
        # Filter out zero values
        D_filtered = D_flat[D_flat > 0]
        S_filtered = S_flat[S_flat > 0]
        
        if len(D_filtered) == 0 or len(S_filtered) == 0:
            return 0.0
        
        # Normalize to 0-1
        D_norm = D_filtered / 100.0
        S_norm = S_filtered / 100.0
        
        try:
            emd = wasserstein_distance(D_norm, S_norm)
            # Convert to similarity score (0-1)
            similarity = np.exp(-5 * emd)
            return similarity
        except:
            return 0.0
    
    def _morphology_similarity(self, D_prime: np.ndarray, S_prime: np.ndarray) -> float:
        """Compute morphological similarity in a bucket"""
        from sklearn.metrics.pairwise import cosine_similarity
        
        if len(D_prime) == 0 or len(S_prime) == 0:
            return 0.0
        
        # Use cosine similarity for morphological features
        sim_matrix = cosine_similarity(D_prime, S_prime)
        return np.mean(sim_matrix)
    
    def _density_similarity(self, D_prime: np.ndarray, S_prime: np.ndarray) -> float:
        """Compute density similarity in a bucket"""
        if len(D_prime) == 0 or len(S_prime) == 0:
            return 0.0
        
        # Use mean absolute percentage similarity
        mean_diff = np.abs(np.mean(D_prime) - np.mean(S_prime)) / np.mean(S_prime)
        similarity = np.exp(-mean_diff)
        return similarity


class AccessibilityPipeline:
    """
    Pipeline for computing accessibility metrics for health and water
    """
    
    def __init__(self, city: str = "accra", target_resolution: int = 30):
        self.city = city
        self.crs = TARGET_CRS
        self.target_resolution = target_resolution
        self.todqa_assessor = TODQAAssessor(k=16)
        
    def load_data(self):
        """Load all required datasets"""
        print("Loading datasets...")
        
        from pathlib import Path
        PROJECT_ROOT = Path("/headless/EXPTS")
        base_dir = PROJECT_ROOT / "New_TODQA" / self.city.lower()

        tif_file1 = "health_accessibility_10\n.tif".strip()
        tif_path1 = f"/headless/EXPTS/New_TODQA/accra/{tif_file1}"

        tif_file2 = "elevation_srtm2014.tif"
        tif_path2 = f"/headless/EXPTS/New_TODQA/accra/{tif_file2}"

        tif_file3 = "accra_worldpop2023.tif"
        tif_path3 = f"/headless/EXPTS/New_TODQA/accra/{tif_file3}"

        # Define paths - ADDED GBA Building Atlas
        paths = {
            "boundary": base_dir / "ashaiman_boundary" / "Ashaiman_boundary.shp",
            "google_obd": base_dir / "google_buildings.geojson",
            "microsoft_obd": base_dir / "microsoft_buildings.geojson",
            "osm_buildings": base_dir / "osm_buildings.gpkg",
            "global_buildings": base_dir / "gba_buildings.gpkg", #"global_buildings.geojson",  # ADDED
            "pipe_network": base_dir / "ashaiman_pipeline" / "pipe_network.shp",
            "health_accessibility": tif_path1,
            "elevation_raster": tif_path2,
            "population_raster": tif_path3,
        }
        
        # Load boundary
        self.boundary = gpd.read_file(paths['boundary']).to_crs(self.crs)
        
        # Create analysis boundary with buffer
        bounds = self.boundary.total_bounds
        centroid = self.boundary.unary_union.centroid
        corners = [Point(bounds[0], bounds[1]), Point(bounds[2], bounds[1]),
                  Point(bounds[0], bounds[3]), Point(bounds[2], bounds[3])]
        distances = [corner.distance(centroid) for corner in corners]
        buffer_distance = max(distances) * 0.1
        self.analysis_boundary = box(*bounds).buffer(buffer_distance)
        self.bounds = self.analysis_boundary.bounds
        
        # Calculate target dimensions
        minx, miny, maxx, maxy = self.bounds
        self.width = int((maxx - minx) / self.target_resolution)
        self.height = int((maxy - miny) / self.target_resolution)
        self.target_transform = rasterio.transform.from_bounds(
            minx, miny, maxx, maxy, self.width, self.height
        )
        
        # Load OBD datasets - ADDED GBA Building Atlas
        self.google_obd = self._load_and_clip(paths['google_obd'])
        self.microsoft_obd = self._load_and_clip(paths['microsoft_obd'])
        self.osm_buildings = self._load_and_clip(paths['osm_buildings'])
        self.global_buildings = self._load_and_clip(paths['global_buildings'])  # ADDED
        
        # Load pipe network
        self.pipe_network = self._load_and_clip(paths['pipe_network'])
        
        # Load rasters
        self.health_access = self._load_and_align_raster(paths['health_accessibility'])
        
        if os.path.exists(paths['elevation_raster']):
            self.elevation = self._load_and_align_raster(paths['elevation_raster'])
        else:
            self.elevation = None
            
        if os.path.exists(paths['population_raster']):
            self.population = self._load_and_align_raster(paths['population_raster'])
        else:
            self.population = None
        
        print(f"Data loaded: Google={len(self.google_obd)}, Microsoft={len(self.microsoft_obd)}, "
              f"OSM={len(self.osm_buildings)}, GBA={len(self.global_buildings)}")
        
        boundary_path = "ranking_results/ashaiman_boundary.geojson"
        self.boundary.to_file(boundary_path, driver='GeoJSON')
        print(f"Boundary saved to {boundary_path}")        

        return self
    
    def _load_and_clip(self, filepath: str) -> gpd.GeoDataFrame:
        """Load and clip vector data"""
        if not os.path.exists(filepath):
            print(f"Warning: File not found: {filepath}")
            return gpd.GeoDataFrame(geometry=[], crs=self.crs)
        
        try:
            gdf = gpd.read_file(filepath)
            if gdf.crs != self.crs:
                gdf = gdf.to_crs(self.crs)
            return gdf[gdf.geometry.intersects(self.analysis_boundary)].copy()
        except Exception as e:
            print(f"Error loading {filepath}: {e}")
            return gpd.GeoDataFrame(geometry=[], crs=self.crs)
    
    def _load_and_align_raster(self, raster_path: str) -> np.ndarray:
        """Load and align raster to target grid"""
        if not os.path.exists(raster_path):
            return None
        
        try:
            with rasterio.open(raster_path) as src:
                transform, width, height = calculate_default_transform(
                    src.crs, self.crs, src.width, src.height,
                    *src.bounds, resolution=self.target_resolution
                )
                
                destination = np.zeros((height, width), dtype=src.dtypes[0])
                reproject(
                    source=rasterio.band(src, 1),
                    destination=destination,
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=self.crs,
                    resampling=Resampling.bilinear
                )
                
                # Clip to analysis bounds
                minx, miny, maxx, maxy = self.bounds
                row_start, col_start = rasterio.transform.rowcol(transform, minx, maxy)
                row_end, col_end = rasterio.transform.rowcol(transform, maxx, miny)
                
                row_start, col_start = max(0, row_start), max(0, col_start)
                row_end, col_end = min(height, row_end), min(width, col_end)
                
                clipped = destination[row_start:row_end, col_start:col_end]
                
                # Resize to target dimensions if needed
                if clipped.shape != (self.height, self.width):
                    from scipy.ndimage import zoom
                    zoom_factors = (self.height/clipped.shape[0], 
                                  self.width/clipped.shape[1])
                    return zoom(clipped, zoom_factors, order=0)
                
                return clipped
        except Exception as e:
            print(f"Error loading raster {raster_path}: {e}")
            return None
    
    def compute_water_accessibility(self) -> Dict[str, np.ndarray]:
        """
        Compute water accessibility using pipe network, terrain, and population density.
        Returns accessibility values for each OBD dataset.
        """
        print("\nComputing water accessibility...")
        
        # Ensure all required data are available
        if self.elevation is None or self.pipe_network is None or self.population is None:
            print("Warning: Missing elevation, pipe network, or population data")
            # Return uniform accessibility as fallback
            uniform_value = np.array([50] * 100)
            return {
                'Google': uniform_value.copy(),
                'Microsoft': uniform_value.copy(),
                'OSM': uniform_value.copy(),
                'GBA': uniform_value.copy()
            }
        
        height, width = self.height, self.width
        from rasterio.features import geometry_mask
        
        # --- 1. Pipe proximity score (linear decay up to 2000 m) ---
        # Create pipe source raster
        pipe_raster = geometry_mask(
            [geom for geom in self.pipe_network.geometry],
            out_shape=(height, width),
            transform=self.target_transform,
            invert=True,
            all_touched=True
        ).astype(float)
        
        # Get coordinates of pipe cells
        y_indices, x_indices = np.where(pipe_raster == 1)
        if len(y_indices) == 0:
            # Fallback: use center point
            center_row, center_col = height // 2, width // 2
            y_indices, x_indices = np.array([center_row]), np.array([center_col])
        
        # Create coordinate arrays for all cells
        rows, cols = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')
        xs, ys = rasterio.transform.xy(self.target_transform, rows.flatten(), cols.flatten())
        coords = np.column_stack([np.array(xs), np.array(ys)])
        
        # Pipe coordinates
        pipe_xs, pipe_ys = rasterio.transform.xy(self.target_transform, y_indices, x_indices)
        pipe_coords = np.column_stack([np.array(pipe_xs), np.array(pipe_ys)])
        
        # Calculate Euclidean distances using KDTree
        tree = cKDTree(pipe_coords)
        distances = np.zeros(len(coords))
        chunk_size = 10000
        for i in range(0, len(coords), chunk_size):
            chunk_end = min(i + chunk_size, len(coords))
            chunk_dist, _ = tree.query(coords[i:chunk_end], k=1)
            distances[i:chunk_end] = chunk_dist
        distance_raster = distances.reshape((height, width))
        
        # Apply linear decay with fixed threshold d_max = 2000 m (from manuscript)
        d_max = 2000.0
        distance_score = np.maximum(0, 1 - distance_raster / d_max)
        
        # --- 2. Terrain slope score (exponential decay) ---
        from scipy.ndimage import sobel
        dz_dx = sobel(self.elevation, axis=1) / (2 * self.target_resolution)
        dz_dy = sobel(self.elevation, axis=0) / (2 * self.target_resolution)
        slope = np.degrees(np.arctan(np.sqrt(dz_dx**2 + dz_dy**2)))
        
        # Apply exponential decay f_s(s) = exp(-0.1 * s) (from manuscript)
        slope_score = np.exp(-0.1 * slope)
        
        # --- 3. Population density score (min-max scaling) ---
        pop = self.population.astype(float)
        p_min = pop.min()
        p_max = pop.max()
        if p_max > p_min:
            population_score = (pop - p_min) / (p_max - p_min)
        else:
            population_score = np.ones_like(pop)  # uniform if no variation
        
        # --- 4. Weighted combination (α=0.5, β=0.3, γ=0.2) ---
        water_accessibility = (0.5 * distance_score +
                            0.3 * slope_score +
                            0.2 * population_score)
        
        # Scale to 0-100 and convert to uint8 for storage
        water_accessibility = (water_accessibility * 100).astype(np.uint8)
        
        # --- 5. Extract values at building centroids for each dataset ---
        results = {}
        for name, buildings in [('Google', self.google_obd),
                                ('Microsoft', self.microsoft_obd),
                                ('OSM', self.osm_buildings),
                                ('GBA', self.global_buildings)]:
            if len(buildings) == 0:
                results[name] = np.array([])
                continue
            
            values = []
            centroids = buildings.geometry.centroid
            for centroid in centroids:
                try:
                    row, col = rasterio.transform.rowcol(self.target_transform,
                                                        centroid.x, centroid.y)
                    if 0 <= row < height and 0 <= col < width:
                        values.append(water_accessibility[row, col])
                    else:
                        values.append(0)
                except:
                    values.append(0)
            results[name] = np.array(values)
        
        # --- 6. Save outputs for mapping ---
        # Create directory if not exists
        os.makedirs("ranking_results", exist_ok=True)
        
        # Save pipe network
        pipe_path = "ranking_results/pipe_network.geojson"
        self.pipe_network.to_file(pipe_path, driver='GeoJSON')
        print(f"Pipe network saved to {pipe_path}")
        
        # Save water accessibility raster
        output_raster_path = "ranking_results/water_accessibility.tif"
        with rasterio.open(
            output_raster_path,
            'w',
            driver='GTiff',
            height=height,
            width=width,
            count=1,
            dtype=water_accessibility.dtype,
            crs=self.crs,                # corrected attribute name
            transform=self.target_transform,   # corrected attribute name
        ) as dst:
            dst.write(water_accessibility, 1)
        print(f"Water accessibility raster saved to {output_raster_path}")
        
        return results
    
    def compute_health_accessibility(self) -> Dict[str, np.ndarray]:
        """
        Compute health accessibility from pre-computed raster
        Returns accessibility values for each OBD dataset
        """
        print("\nComputing health accessibility...")
        
        if self.health_access is None:
            print("Warning: Health accessibility raster not found")
            uniform_value = np.array([50] * 100)
            return {
                'Google': uniform_value.copy(),
                'Microsoft': uniform_value.copy(),
                'OSM': uniform_value.copy(),
                'GBA': uniform_value.copy()  # ADDED
            }
        
        # Scale health accessibility to 0-100
        health_raster = self.health_access
        if health_raster.max() > 0:
            health_raster = (health_raster / health_raster.max() * 100).astype(np.uint8)
        
        height, width = health_raster.shape
        results = {}
        
        # ADDED Global Building Atlas
        for name, buildings in [('Google', self.google_obd),
                               ('Microsoft', self.microsoft_obd),
                               ('OSM', self.osm_buildings),
                               ('GBA', self.global_buildings)]:
            if len(buildings) == 0:
                results[name] = np.array([])
                continue
            
            values = []
            centroids = buildings.geometry.centroid
            
            for centroid in centroids:
                try:
                    row, col = rasterio.transform.rowcol(self.target_transform, 
                                                        centroid.x, centroid.y)
                    if 0 <= row < height and 0 <= col < width:
                        values.append(health_raster[row, col])
                    else:
                        values.append(0)
                except:
                    values.append(0)
            
            results[name] = np.array(values)
        
        return results
    
    def generate_synthetic_reference(self, accessibility_values: np.ndarray, 
                                   n_samples: int = 1000) -> np.ndarray:
        """
        Generate synthetic reference samples for TODQA
        Based on the distribution of accessibility values
        """
        if len(accessibility_values) == 0:
            return np.array([])
        
        # Filter out zero values
        valid_values = accessibility_values[accessibility_values > 0]
        
        if len(valid_values) == 0:
            return np.array([])
        
        # Sample from the distribution
        n_samples = min(n_samples, len(valid_values))
        
        # Use kernel density estimation or simple random sampling
        if len(valid_values) >= n_samples:
            indices = np.random.choice(len(valid_values), n_samples, replace=False)
            synthetic = valid_values[indices]
        else:
            # If not enough samples, repeat with some noise
            repeats = n_samples // len(valid_values) + 1
            synthetic = np.tile(valid_values, repeats)[:n_samples]
            # Add small noise
            synthetic = synthetic + np.random.normal(0, 1, n_samples)
            synthetic = np.clip(synthetic, 0, 100).astype(np.uint8)
        
        return synthetic


class RankAggregation:
    """
    2-Approximation Algorithm for Rank Aggregation
    Based on Li et al. (2019) methodology
    """
    
    def __init__(self):
        self.rankings = {}
        self.weights = {}
        
    def add_ranking(self, task_name: str, ranking: Dict[str, float], weight: float = 1.0):
        """
        Add a ranking for a specific task
        
        Parameters:
        -----------
        task_name : str, name of the task (e.g., 'health', 'water')
        ranking : dict, mapping dataset names to scores (higher is better)
        weight : float, weight for this task in aggregation
        """
        self.rankings[task_name] = ranking
        self.weights[task_name] = weight
        
    def compute_aggregated_ranking(self) -> Tuple[List[str], Dict[str, float]]:
        """
        Compute aggregated ranking using weighted score summation
        
        Returns:
        --------
        sorted_datasets : list, datasets sorted by aggregated score (best first)
        aggregated_scores : dict, final aggregated scores for each dataset
        """
        
        if not self.rankings:
            raise ValueError("No rankings added")
        
        # Get all unique datasets
        all_datasets = set()
        for ranking in self.rankings.values():
            all_datasets.update(ranking.keys())
        all_datasets = list(all_datasets)
        
        # Line 1: Initialize score vector r[dataset] ← 0 for each dataset
        r = {dataset: 0.0 for dataset in all_datasets}
        
        # Lines 2-6: For each dataset, for each task, add weighted score
        for dataset in all_datasets:  # Line 2
            for task_name, ranking in self.rankings.items():  # Line 3
                weight = self.weights.get(task_name, 1.0)
                # τ_d(t) is the score of dataset d for task t
                # Use 0 if dataset not in this task's ranking
                task_score = ranking.get(dataset, 0.0)
                # Line 4: r[d] ← r[d] + μ_t × τ_d(t)
                r[dataset] += weight * task_score  # Line 4
            # Line 5-6: End for loops
        
        # Line 7: Sort datasets by descending values of r[d]
        # Create list of (dataset, score) tuples
        dataset_scores = list(r.items())
        # Sort by score in descending order (higher score = better)
        sorted_dataset_scores = sorted(dataset_scores, key=lambda x: x[1], reverse=True)
        
        # Extract sorted dataset names
        sorted_datasets = [dataset for dataset, _ in sorted_dataset_scores]
        
        # Return aggregated scores dictionary
        aggregated_scores = {dataset: score for dataset, score in sorted_dataset_scores}
        
        # Line 8: Return σ (represented as sorted_datasets)
        return sorted_datasets, aggregated_scores
    
    def compute_task_relevancy_scores(self, assessor,
                                     D_features: Dict[str, np.ndarray],
                                     S_features: Dict[str, np.ndarray],
                                     task_type: str = 'accessibility') -> Dict[str, float]:
        """
        Compute task relevancy scores for all datasets using TODQA
        
        Parameters:
        -----------
        assessor : TODQAAssessor instance
        D_features : dict mapping dataset names to feature arrays
        S_features : dict mapping task names to synthetic reference feature arrays
        task_type : type of similarity function to use
        
        Returns:
        --------
        scores : dict mapping dataset names to task relevancy scores
        """
        scores = {}
        
        for dataset_name, features in D_features.items():
            if len(features) == 0:
                scores[dataset_name] = 0.0
                continue
            
            # Average score across all task references
            task_scores = []
            for task_name, s_features in S_features.items():
                if len(s_features) > 0:
                    q_score = assessor.compute_relevancy(
                        features.reshape(-1, 1),  # Accessibility is 1D feature
                        s_features.reshape(-1, 1),
                        task_type=task_type
                    )
                    task_scores.append(q_score)
            
            scores[dataset_name] = np.mean(task_scores) if task_scores else 0.0
        
        return scores


In [5]:
def main():
    """
    Main execution pipeline:
    1. Compute health and water accessibility for four OBD datasets
    2. Generate synthetic references
    3. Compute task relevancy scores using TODQA
    4. Aggregate rankings using 2-Approximation Algorithm
    """
    
    print("=" * 70)
    print("OBD DATASET RANKING PIPELINE")
    print("Datasets: Google, Microsoft, OSM, GlobalBuildingMap")
    print("Tasks: 1. Health Facility Accessibility, 2. Clean Water Accessibility")
    print("=" * 70)
    
    # Initialize pipeline
    pipeline = AccessibilityPipeline(city="accra", target_resolution=30)
    
    # Step 1: Load data (now includes Global Building Atlas)
    print("\nStep 1: Loading data...")
    pipeline.load_data()
    
    # Step 2: Compute accessibility metrics for all four datasets
    print("\nStep 2: Computing accessibility metrics...")
    
    # Health accessibility
    health_results = pipeline.compute_health_accessibility()
    print(f"  Health accessibility computed:")
    for dataset, values in health_results.items():
        if len(values) > 0:
            print(f"    {dataset}: {len(values)} buildings, mean={values.mean():.1f}")
        else:
            print(f"    {dataset}: No buildings found")
    
    # Water accessibility
    water_results = pipeline.compute_water_accessibility()
    print(f"  Water accessibility computed:")
    for dataset, values in water_results.items():
        if len(values) > 0:
            print(f"    {dataset}: {len(values)} buildings, mean={values.mean():.1f}")
        else:
            print(f"    {dataset}: No buildings found")
    
    # Step 3: Generate synthetic references
    print("\nStep 3: Generating synthetic references...")
    
    # Combine all accessibility values to create reference distribution
    all_health_values = np.concatenate([v for v in health_results.values() if len(v) > 0])
    all_water_values = np.concatenate([v for v in water_results.values() if len(v) > 0])
    
    health_reference = pipeline.generate_synthetic_reference(all_health_values, 1000)
    water_reference = pipeline.generate_synthetic_reference(all_water_values, 1000)
    
    print(f"  Synthetic references: Health={len(health_reference)}, Water={len(water_reference)}")
    
    # Step 4: Compute task relevancy scores using TODQA
    print("\nStep 4: Computing task relevancy scores (TODQA)...")
    
    # Prepare feature dictionaries for all four datasets
    D_features_health = {
        'Google': health_results.get('Google', np.array([])),
        'Microsoft': health_results.get('Microsoft', np.array([])),
        'OSM': health_results.get('OSM', np.array([])),
        'GBA': health_results.get('GBA', np.array([]))  # ADDED
    }
    
    D_features_water = {
        'Google': water_results.get('Google', np.array([])),
        'Microsoft': water_results.get('Microsoft', np.array([])),
        'OSM': water_results.get('OSM', np.array([])),
        'GBA': water_results.get('GBA', np.array([]))  # ADDED
    }
    
    S_features = {
        'health': health_reference,
        'water': water_reference
    }
    
    # Initialize TODQA assessor
    todqa = TODQAAssessor(k=16)
    
    # Compute health accessibility scores for all four datasets
    health_scores = {}
    for dataset, features in D_features_health.items():
        if len(features) > 0 and len(health_reference) > 0:
            q_score = todqa.compute_relevancy(
                features.reshape(-1, 1),
                health_reference.reshape(-1, 1),
                task_type='accessibility'
            )
            health_scores[dataset] = q_score
        else:
            health_scores[dataset] = 0.0
    
    print("  Health accessibility relevancy scores:")
    for dataset, score in health_scores.items():
        print(f"    {dataset}: {score:.4f}")
    
    # Compute water accessibility scores for all four datasets
    water_scores = {}
    for dataset, features in D_features_water.items():
        if len(features) > 0 and len(water_reference) > 0:
            q_score = todqa.compute_relevancy(
                features.reshape(-1, 1),
                water_reference.reshape(-1, 1),
                task_type='accessibility'
            )
            water_scores[dataset] = q_score
        else:
            water_scores[dataset] = 0.0
    
    print("  Water accessibility relevancy scores:")
    for dataset, score in water_scores.items():
        print(f"    {dataset}: {score:.4f}")
    
    # Step 5: Aggregate rankings using 2-Approximation Algorithm
    print("\nStep 5: Aggregating rankings (2-Approximation Algorithm)...")
    
    rank_aggregator = RankAggregation()
    
    # Add rankings with weights
    health_weight = 0.5
    water_weight = 0.5
    
    rank_aggregator.add_ranking('health_accessibility', health_scores, health_weight)
    rank_aggregator.add_ranking('water_accessibility', water_scores, water_weight)
    
    # Compute aggregated ranking for all four datasets
    sorted_datasets, aggregated_scores = rank_aggregator.compute_aggregated_ranking()
    
    # Step 6: Display results
    print("\n" + "=" * 70)
    print("FINAL RANKING RESULTS")
    print("=" * 70)
    
    print("\nIndividual Task Rankings:")
    print("-" * 40)
    
    print("\n1. Health Facility Accessibility:")
    health_ranked = sorted(health_scores.items(), key=lambda x: x[1], reverse=True)
    for rank, (dataset, score) in enumerate(health_ranked, 1):
        print(f"   {rank}. {dataset}: {score:.4f}")
    
    print("\n2. Clean Water Accessibility:")
    water_ranked = sorted(water_scores.items(), key=lambda x: x[1], reverse=True)
    for rank, (dataset, score) in enumerate(water_ranked, 1):
        print(f"   {rank}. {dataset}: {score:.4f}")
    
    print("\n" + "-" * 40)
    print("Aggregated Ranking (2-Approximation Algorithm):")
    print("-" * 40)
    
    for rank, dataset in enumerate(sorted_datasets, 1):
        score = aggregated_scores[dataset]
        print(f"   {rank}. {dataset}: aggregated score = {score:.4f}")
    
    # Calculate and display final normalized scores
    print("\n" + "-" * 40)
    print("Final Normalized Scores (0-1):")
    print("-" * 40)
    
    # Normalize aggregated scores to 0-1 range
    min_score = min(aggregated_scores.values())
    max_score = max(aggregated_scores.values())
    
    if max_score != min_score:
        normalized_scores = {
            dataset: (score - min_score) / (max_score - min_score)
            for dataset, score in aggregated_scores.items()
        }
    else:
        normalized_scores = {dataset: 1.0 for dataset in aggregated_scores.keys()}
    
    for dataset, score in sorted(normalized_scores.items(), key=lambda x: x[1], reverse=True):
        print(f"   {dataset}: {score:.4f}")
    
    # Step 7: Save results
    print("\nSaving results...")
    results = {
        'task_scores': {
            'health_accessibility': health_scores,
            'water_accessibility': water_scores
        },
        'weights': {
            'health_accessibility': health_weight,
            'water_accessibility': water_weight
        },
        'aggregated_ranking': sorted_datasets,
        'aggregated_scores': aggregated_scores,
        'normalized_scores': normalized_scores
    }
    
    os.makedirs("ranking_results", exist_ok=True)
    with open("ranking_results/obd_ranking_results.json", "w") as f:
        json.dump(results, f, indent=2)
    
    # Save detailed accessibility values
    accessibility_data = {
        'health_values': {k: v.tolist() for k, v in health_results.items() if len(v) > 0},
        'water_values': {k: v.tolist() for k, v in water_results.items() if len(v) > 0},
        'synthetic_references': {
            'health': health_reference.tolist() if len(health_reference) > 0 else [],
            'water': water_reference.tolist() if len(water_reference) > 0 else []
        }
    }
    
    with open("ranking_results/accessibility_values.json", "w") as f:
        json.dump(accessibility_data, f, indent=2)
    
    print("\n" + "=" * 70)
    print("PIPELINE COMPLETED SUCCESSFULLY!")
    print(f"Results saved to 'ranking_results' directory")
    print("=" * 70)
    
    return results


if __name__ == "__main__":
    # Run main pipeline
    results = main()

OBD DATASET RANKING PIPELINE
Datasets: Google, Microsoft, OSM, GlobalBuildingMap
Tasks: 1. Health Facility Accessibility, 2. Clean Water Accessibility

Step 1: Loading data...
Loading datasets...


/tmp/ipykernel_1450097/640231440.py:195: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = self.boundary.unary_union.centroid


Data loaded: Google=91331, Microsoft=44695, OSM=31404, GBA=68090
Boundary saved to ranking_results/ashaiman_boundary.geojson

Step 2: Computing accessibility metrics...

Computing health accessibility...
  Health accessibility computed:
    Google: 91331 buildings, mean=43.5
    Microsoft: 44695 buildings, mean=42.1
    OSM: 31404 buildings, mean=50.1
    GBA: 68090 buildings, mean=44.8

Computing water accessibility...
Pipe network saved to ranking_results/pipe_network.geojson


AttributeError: 'AccessibilityPipeline' object has no attribute 'target_crs'

In [7]:
results

{'task_scores': {'health_accessibility': {'Google': np.float64(0.9726593385746379),
   'Microsoft': np.float64(0.9204352781701027),
   'OSM': np.float64(0.7268322122880696),
   'Global': np.float64(0.8322988661317204)},
  'water_accessibility': {'Google': np.float64(0.9753663515705194),
   'Microsoft': np.float64(0.9428226666663616),
   'OSM': np.float64(0.8947612805457444),
   'Global': np.float64(0.8696571161434651)}},
 'weights': {'health_accessibility': 0.5, 'water_accessibility': 0.5},
 'aggregated_ranking': ['Google', 'Microsoft', 'Global', 'OSM'],
 'aggregated_scores': {'Google': np.float64(0.9740128450725787),
  'Microsoft': np.float64(0.9316289724182322),
  'Global': np.float64(0.8509779911375928),
  'OSM': np.float64(0.810796746416907)},
 'normalized_scores': {'Google': np.float64(1.0),
  'Microsoft': np.float64(0.7403205137027474),
  'Global': np.float64(0.24618432281887817),
  'OSM': np.float64(0.0)}}